# Tutorial 1: Basic Use Cases

In this tutorial, we are going to see how we can use **RadPrompter** for simplified and reproducible LLM prompting. We will cover more advanced capabilities in the next tutorials.

RadPrompter uses three simple components:

1. **Prompt**: The `Prompt` is the core message-passing recipe between the user and the model.
2. **Client**: The `Client` is responsible for contacting various LLM servers.
3. **Engine**: The `Engine` is the core functionality that ties the other two components together and coordinates running a job.

## Prompt

Let's start with the `Prompt`. We first have to import it from the **RadPrompter** package.

In [2]:
from radprompter import Prompt

Prompts can be initiated by a toml file. Let's read `01_Basic-Usecase.toml` to see its structure:

In [3]:
with open("./01_Basic-Usecase.toml", "r") as f:
    lines = f.readlines()
print("".join(lines))

[METADATA]

version = 0.1
description = "A sample prompt for RadPrompter"


[CONSTRUCTOR]
system = "You are an experienced radiologist that help users extract infromation from radiology reports."
user = """Does the following report indicate a normal or abnormal finding?
{{report}}

Just reply with "normal" or "abnormal" to indicate your answer, without any additional information.
"""


As you can see, the minimum requirements for a prompt file are to have a `[METADATA]` and a `[CONSTRUCTOR]` section. The metadata is used for version tracking and verbose communication. The constructor is used to craft the LLM chat structure.

In this example, we specify a `system` attribute which will be the system prompt for the LLM. We also have a `user` attribute which contains the user's request. You might notice the `{{report}}` tag being enclosed in double curly brackets. This denotes a **placeholder** item that can be replaced with your actual report. Note that this curly bracket placeholder notation can be mixed with any arbitrary variable name that you like, and "report" is just an example. We will learn more about this as we move forward.

Let's now create our prompt:

In [4]:
prompt = Prompt("./01_Basic-Usecase.toml")
prompt

As you can see, each prompt comes with a tabular visualization for easier interpretation. The placeholders will be in a light orange color, and the assistant's response will replace the blue `[... response ...]` tags.

## Client

RadPrompter supports several LLM clients out of the box, including:

- `OpenAIClient`: For accessing OpenAI's models
- `vLLMClient`: For accessing open-source LLMs like Llama hosted using the [vLLM package](https://vllm.ai/).
- `OllamaClient`: For accesing [Ollama](https://ollama.com/) open source models.

To instantiate a client, you need to provide the following:

1. `model` [Required]: The name of the model to use (e.g., "gpt-3.5-turbo" for OpenAI,  "meta-llama/Meta-Llama-3-8B-Instruct" for vLLM and "phi3" for Ollama).
2. `base_url` [Optional]: The URL of the REST API endpoint.
3. `api_key` [Optional]: Your API key for the service. If not provided, the client will attempt to read it from an environment variable (in case of `OpenAIClient`) or set that to "EMPTY" (`vLLMClient` and `OllamaClient`).
4. `temperature` [Optional]: Sampling temperature for the LLM (Default: `0.0`).
5. `seed` [Optional]: Sampling seed for the LLM (Default: `42`).

Here's an example of instantiating a vLLMClient:

In [5]:
from radprompter import vLLMClient

client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:9999/v1",
    temperature = 0.0,
    seed=42
)

Now that we have set up our client, we need to coonect everything together using the RadPrompter **Engine**.

## Engine

The `RadPrompter` class is the engine that ties everything together. To use it, we need to provide:

1. `client`: The LLM client to use 
2. `prompt`: The prompt object
3. `output_file`: The path to save the results to (must be a .csv file)
4. `concurrency`: As we are hitting a server with requests, you can batch your requests together for faster processing (Default: `1`). 

Let's instantiate the engine:

In [6]:
from radprompter import RadPrompter

engine = RadPrompter(
    client=client,
    prompt=prompt, 
    output_file="output_tutorial_1.csv",
    concurrency=2,
)

To run the engine, we pass it a list of dictionaries. Each dictionary should contain keys matching the placeholders in the prompt. Any additional keys will be included in the output file. 

So let's create a list of our inputs:

In [7]:
import glob

report_files = glob.glob("../../sample_reports/*.txt")

reports = []
for file in report_files:
    with open(file, "r") as f:
        reports.append({"report": f.read(), "file_name": file})
        
reports[0]

{'report': 'Clinical Information:\n72-year-old female with sudden onset shortness of breath and hypoxia. Evaluate for pulmonary embolism.\nTechnique:\nCT pulmonary angiography with IV contrast was performed.\nFindings:\nThere is a large saddle embolus extending from the main pulmonary artery into the right and left main pulmonary arteries. Multiple filling defects are also seen within the segmental and subsegmental branches bilaterally, indicating additional pulmonary emboli.\nThe right ventricle appears dilated at 4.6 cm, greater than the left ventricle. There is flattening of the interventricular septum, suggesting right heart strain.\nMosaic perfusion is present within the lungs bilaterally, likely representing pulmonary infarcts.\nSmall bilateral pleural effusions are noted.\nImpression:\n\nSaddle pulmonary embolism involving the main pulmonary artery, right and left main branches, and multiple segmental/subsegmental branches.\nFindings of right heart strain with right ventricular 

Now we have a list of 3 reports, each having a `report` key (which on every run will replace the `{{report}}` placeholder in our prompt) and a `file_name` key that will be included in the output csv file for report identification.

Now let's run our `engine`:

In [8]:
engine(reports)

Processing items:   0%|          | 0/3 [00:02<?, ?it/s]


APIConnectionError: Connection error.

The engine will process each report and save the results to `output_tutorial_1.csv`.

In [ ]:
import pandas as pd

df = pd.read_csv("output_tutorial_1.csv", index_col='index')
df

,default_response,report,file_name
index,,,
0,Abnormal,Clinical Information:\n72-year-old female with...,../../sample_reports/sample_report_2.txt
1,Abnormal,Here is an example radiology report describing...,../../sample_reports/sample_report_3.txt
2,Abnormal,Clinical Information:\n67-year-old male with s...,../../sample_reports/sample_report_1.txt


The column `default_response` will contain the LLM output. In future tutorials, you will learn how to use **Schemas** to further customize your output.

We can also save a log of the run using the `save_log` method:

In [ ]:
engine.save_log("log_tutorial_1.log")

with open("log_tutorial_1.log", "r") as f:
    print(f.read())

RadPrompter Version: 1.1.0
Model: meta-llama/Meta-Llama-3-8B-Instruct
Prompt TOML: /Users/bardiakhosravi/Desktop/GitHub/RadPrompter/tutorials/01_Basic-Usecase/01_Basic-Usecase.toml
Prompt Version: 0.1
Prompt Hash: ce8b273997755ad7279e9c7a13337c70
Concurrency Factor: 2
Start Time: 2024-05-19 16:43:25
End Time: 2024-05-19 16:43:26
Duration: 1.0
Number of Items: 3
Average Processing Time: 0.3333333333333333


-------------------- *** - Prompt Content - *** --------------------
[METADATA]

version = 0.1
description = "A sample prompt for RadPrompter"


[CONSTRUCTOR]
system = "You are an experienced radiologist that help users extract infromation from radiology reports."
user = """Does the following report indicate a normal or abnormal finding?
{{report}}

Just reply with "normal" or "abnormal" to indicate your answer, without any additional information.
"""


This will save detailed information about the run, including the prompt used, to the specified file.

And that's it for the basics of using RadPrompter! In the next tutorials we'll cover more advanced features.